In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


In [2]:
test = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")
train = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")

In [3]:
X_train = train.drop('Made_Purchase', axis='columns')
Y_train = train['Made_Purchase']

X_test=test


In [4]:
import warnings
warnings.filterwarnings('ignore')

num_imputed = X_train.select_dtypes(include=np.number)
cat_imputed = X_train.select_dtypes(include=["object"])

test_num_imputed = X_test.select_dtypes(include=np.number)
test_cat_imputed = X_test.select_dtypes(include=["object"])

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

KNN = KNNImputer(n_neighbors=10, missing_values=np.nan)
SI = SimpleImputer(missing_values=np.nan,strategy="most_frequent")

num_imputed = pd.DataFrame(KNN.fit_transform(num_imputed),columns=num_imputed.columns)
cat_imputed = pd.DataFrame(SI.fit_transform(cat_imputed),columns=cat_imputed.columns)

test_num_imputed = pd.DataFrame(KNN.transform(test_num_imputed),columns=test_num_imputed.columns)
test_cat_imputed = pd.DataFrame(SI.transform(test_cat_imputed),columns=test_cat_imputed.columns)

In [6]:
for col in ['OS', 'SearchEngine','Zone', 'Type of Traffic']:
    num_imputed[col] = num_imputed[col].astype('object')
    
    
for col in ['OS', 'SearchEngine','Zone', 'Type of Traffic']:
    test_num_imputed[col] = test_num_imputed[col].astype('object')

train_encoded_cat_col=num_imputed.iloc[:,10:14]

test_encoded_cat_col=test_num_imputed.iloc[:,10:14]

In [7]:
num_imputed = num_imputed.drop(["OS","SearchEngine",'Zone',"Type of Traffic"],axis='columns')
test_num_imputed = test_num_imputed.drop(["OS","SearchEngine",'Zone',"Type of Traffic"],axis='columns')

In [8]:
from sklearn.preprocessing import MaxAbsScaler
MAS=MaxAbsScaler()

MAS.fit(num_imputed)
num_scaled = pd.DataFrame(MAS.transform(num_imputed),columns=num_imputed.columns)
test_num_scaled=pd.DataFrame(MAS.transform(test_num_imputed),columns=test_num_imputed.columns)


train_num=pd.concat((train_encoded_cat_col,num_scaled),axis=1,join="inner")
test_num=pd.concat((test_encoded_cat_col,test_num_scaled),axis=1,join="inner")

In [9]:
for cols in ['OS', 'SearchEngine','Zone', 'Type of Traffic']:
    train_num[cols] = train_num[cols].astype('float64')
    
    
for cols in ['OS', 'SearchEngine','Zone', 'Type of Traffic']:
    test_num[cols] = test_num[cols].astype('float64')


In [10]:
X_train=pd.concat((train_num,cat_imputed),axis=1,join="inner")
X_test=pd.concat((test_num,test_cat_imputed),axis=1,join="inner")

In [11]:
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

LE = LabelEncoder()

object_col = X_train.select_dtypes(include=np.object).columns.tolist()
print(object_col)
# Encode labels in column 'species'.
for col in object_col:
    X_train[col] = LE.fit_transform(X_train[col])
    

['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies Setting', 'Education', 'Marital Status']


In [12]:
import warnings
warnings.filterwarnings('ignore')

object_col = X_test.select_dtypes(include=np.object).columns.tolist()
print(object_col)
# Encode labels in column 'species'.
for col in object_col:
    X_test[col] = LE.fit_transform(X_test[col])


['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies Setting', 'Education', 'Marital Status']


In [13]:
X_train

,OS,SearchEngine,Zone,Type of Traffic,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,...,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,WeekendPurchase,Month_SeasonalPurchase,CustomerType,Gender,Cookies Setting,Education,Marital Status
0,4.0,1.0,9.0,3.0,0.000000,0.000000,0.000000,0.000000,0.001418,0.000000,...,1.000000,0.0,0.0,0.0,2,2,2,1,2,1
1,3.0,2.0,2.0,4.0,0.000000,0.000000,0.000000,0.000000,0.002837,0.000042,...,0.700000,0.0,0.0,0.0,2,2,0,1,3,0
2,3.0,3.0,1.0,4.0,0.000000,0.000000,0.000000,0.000000,0.014184,0.009809,...,0.250000,0.0,0.0,1.0,2,2,0,0,3,0
3,2.0,4.0,3.0,3.0,0.000000,0.000000,0.000000,0.000000,0.001418,0.000000,...,1.000000,0.0,0.4,0.0,2,2,1,0,0,2
4,1.0,2.0,1.0,5.0,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.0,0.0,1.0,2,2,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14726,3.0,2.0,6.0,3.0,0.037037,0.001177,0.000000,0.000000,0.055319,0.015368,...,0.087995,0.0,0.0,0.0,7,2,0,1,3,0
14727,2.0,2.0,2.0,4.0,0.296296,0.034431,0.083333,0.022358,0.015603,0.003953,...,0.055392,0.0,0.0,0.0,6,2,2,0,0,0
14728,2.0,2.0,4.0,2.0,0.074074,0.022243,0.083333,0.256063,0.014184,0.017877,...,0.116667,0.0,0.0,0.0,0,2,2,2,3,2
14729,2.0,4.0,4.0,1.0,0.000000,0.000000,0.000000,0.000000,0.008511,0.016522,...,0.166667,0.0,0.0,0.0,5,2,2,2,2,0


In [14]:
Y_train

0        False
1        False
2        False
3        False
4        False
         ...  
14726     True
14727     True
14728     True
14729     True
14730     True
Name: Made_Purchase, Length: 14731, dtype: bool

In [15]:
X_test

,OS,SearchEngine,Zone,Type of Traffic,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,...,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,WeekendPurchase,Month_SeasonalPurchase,CustomerType,Gender,Cookies Setting,Education,Marital Status
0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.001418,0.000000,...,1.000000,0.0,0.0,0.0,2,2,2,1,2,2
1,2.0,2.0,1.0,2.0,0.000000,0.000000,0.000000,0.000000,0.002837,0.001000,...,0.500000,0.0,0.0,0.0,2,2,2,0,1,0
2,2.0,2.0,1.0,3.0,0.000000,0.000000,0.000000,0.000000,0.026950,0.002411,...,0.122807,0.0,0.0,0.0,2,2,0,2,0,1
3,2.0,2.0,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.002837,0.000578,...,0.500000,0.0,0.8,0.0,2,2,2,2,1,1
4,1.0,1.0,4.0,3.0,0.000000,0.000000,0.000000,0.000000,0.022695,0.006374,...,0.129167,0.0,0.4,0.0,2,2,0,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6594,4.0,1.0,1.0,5.0,0.000000,0.000000,0.000000,0.000000,0.009929,0.003251,...,0.142857,0.0,0.0,1.0,2,2,2,2,1,1
6595,2.0,4.0,9.0,11.0,0.000000,0.000000,0.125000,0.017259,0.253901,0.027175,...,0.129988,0.0,0.0,0.0,0,2,1,2,2,1
6596,8.0,5.0,1.0,2.0,0.185185,0.029177,0.041667,0.010591,0.046809,0.014438,...,0.045635,0.0,0.6,0.0,6,2,0,2,1,0
6597,2.0,2.0,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.004255,0.000141,...,0.666667,0.0,0.0,1.0,6,2,2,1,0,1


**MODEL**

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_train,Y_train,test_size=0.2,random_state=42)

**DECISION-TREE**

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

DTC=DecisionTreeClassifier()

DTC.fit(x_train,y_train)
y_pred=DTC.predict(x_test)
print(classification_report(y_test,y_pred))
print("Accuracy:",accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.59      0.58      0.58      1885
        True       0.28      0.29      0.28      1062

    accuracy                           0.47      2947
   macro avg       0.43      0.43      0.43      2947
weighted avg       0.48      0.47      0.48      2947

Accuracy: 0.47268408551068886


**RANDOM-FORREST**

In [18]:
from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier()
RFC.fit(x_train,y_train)
y_pred=RFC.predict(x_test)
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test, y_pred))


Accuracy: 0.5290125551408211


**LOGISTIC REGRESSION**

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred=logreg.predict(x_test)
print(classification_report(y_test,y_pred))
print("Accuracy:",accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.67      0.97      0.79      1885
        True       0.74      0.17      0.28      1062

    accuracy                           0.68      2947
   macro avg       0.71      0.57      0.54      2947
weighted avg       0.70      0.68      0.61      2947

Accuracy: 0.67967424499491


**ADABOOST**

In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [21]:
model = AdaBoostClassifier()


In [22]:
import warnings
warnings.filterwarnings('ignore')

parames = {
    'n_estimators': [100,200,300],
    'learning_rate' : [0.0001, 0.001, 0.01]
    }
grid_search_ada= GridSearchCV(model, parames , cv=5 , scoring = 'neg_mean_squared_error', return_train_score=True)

grid_search_ada.fit(X_train,Y_train)

grid_search_ada.best_params_ 

{'learning_rate': 0.0001, 'n_estimators': 100}

In [23]:
model = AdaBoostClassifier(n_estimators=100, learning_rate=0.0001, random_state=0)


In [24]:
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.6837461825585341


In [25]:
model.fit(X_train, Y_train)

prediction_test = model.predict(X_test)
prediction_test

array([False, False, False, ..., False, False, False])

In [26]:
submission=pd.DataFrame(columns=['id', 'Made_Purchase'])
submission['id']=[i for i in range(len(prediction_test))]
submission["Made_Purchase"]=[i for i in prediction_test]
submission.to_csv('submission.csv',index=False)
submission

,id,Made_Purchase
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False
...,...,...
6594,6594,False
6595,6595,False
6596,6596,False
6597,6597,False
